### 번역자: 고려대학교 수학과(12) 최시현
[AiKorea 번역 프로젝트](https://github.com/aikorea/cs231n)를 이어받아서 진행하였습니다. 기존 AiKorea에서 이미 번역되어있는 CourseNote의 경우 유사한 부분이 많습니다. <br /> 
(깨진 삽화를 추가하거나 개인적으로 번역이 매끄럽지 못하다 생각하는 부분은 수정하였습니다.) <br />
문제가 될시에는 자진 삭제하겠습니다. 오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

## Learning

이전 섹션들에서는 뉴럴 네트워크의 정적인 부분들에 대하여 논의하였다. <br /> 
네트워크 연결성, 데이터, 손실함수를 설정할지에 대해 알아보았다. <br />
이번 섹션에서는 동적인 부분들을 소개한다. 즉, 파라미터들을 학습하는 과정과 좋은 하이퍼 파라미터를 찾는 과정을 학습한다.

## 그라디언트 체크

이론적으로 그라디언트 체크라 하면, 수치적으로 계산한 그라디언트와 수식으로 계산한 그라디언트를 비교하는 정도라 매우 간단하다. <br />
하지만 실전에서 이 과정은 생각했던것 보다 복잡하고 오류가 나기 쉽다. <br />
여기서 팁, 트릭, 주의해야할 이슈들을 소개하고자 한다.

#### Use the centered formula

수치적 그라디언트를 사용할때 아마 finite difference 근사를 보았을것이다.

<img src="./Images/1.png" width=300 />

여기서 h는 아주 작은 수이고, 실전에서 보통 ${e}^{-5}$정도의 수를 사용한다. <br /> 
하지만 centerd difference 공식을 사용하는 것이 더 나은것으로 나타난다.

<img src="./Images/2.png" width=300 />

이 공식은 $f(x+h)$ 말고도 $f(x-h)$도 계산하여야 하므로 최초식보다 계산량이 두 배 많지만 훨씬 정확한 근사를 제공한다. <br />
$f(x+h)$ 및 $f(x-h)$의 ($x$근방에서의) 테일러 전개를 고려하면 이유를 금방 알 수 있다. 척 식은 $O(h)$의 오차가 있는데 반해, <br />
두 번째 식은 오차가 $O({h}^{2})$이다. (즉, 이차 근사이다)

#### 상대 오차를 사용하라

그라디언트의 참값 ${ f }_{ a }^{\prime }$와 수치적 근사값 ${ f }_{ n }^{\prime }$을 비교하려면 어떤 디테일을 점검하여야 할까? <br />
이 둘이 비슷하지 않음을 어떻게 알아낼 수 있을까? <br />
가장 쉽게는 둘의 절대 오차$|{ f }_{ a }^{\prime } - { f }_{ n }^{\prime }|$ 혹은 그 제곱을 추적하여 이 값들이 언제가 어느 한계점을 넘으면 그라디언트 오류라 할 수 있겠다. <br />
그렇지만 절대 오차에는 문제가 있는 것이, 가령 절대 오차가 ${e}^{-4}$라 가정해 보자. <br /> 
만약 ${ f }_{ a }^{\prime }$ 와 ${ f }_{ n }^{\prime }$ 가 모두 1.0 언저리라면 ${e}^{-4}$의 오차 정도는 매우 훌륭한 근사이고 ${ f }_{ a }^{\prime } \approx { f }_{ n }^{\prime }$이라 할 수 있다. <br />
그런데 만약 두 그라디언트가 ${e}^{-5}$거나 더 작은 값이라면? 그렇다면 ${e}^{-4}$는 매우 큰 차이가 되고 근사가 실패했다고 보아야 한다. <br />
따라서 절대 오차와 두 그라디언트 값의 비율을 고려하는 상대오차가 언제나 더 적절하다.

<img src="./Images/3.png" width=200 />

보통 상대 오차 공식은 분모에 ${ f }_{ a }^{\prime }$ 혹은 ${ f }_{ n }^{\prime }$ 둘 중 하나만 있지만, 둘 중 최대값을 분모로 사용하는 것이 좋다. <br />
그래야 공식의 대칭성이 생기고 둘 중 하나가 정확히 0이 되어 분모가 0이 되는 사태를 방지할 수 있다. (ReLU를 사용하면 자주 일어나는 문제이다) <br />
${ f }_{ a }^{\prime }$와 ${ f }_{ n }^{\prime }$ 가 모두 정확이 0이 된다면? <br />
이 때는 상대 오차를 점검할 필요 없이 그라디언트 체크를 통과하여야 한다. 나의 코드가 이 상황을 감안하여 짜여진 코드인지 점검해보아야 한다.

#### 실제 상황에서의 유용한 가이드
- (상대 오차) > ${e}^{-2}$이면 그라디언트 계산이 아마 잘못되었을 수도 있다.
- ${e}^{-2}$ (상대 오차) > ${e}^{-4}$ 면 불편함을 느끼기 바란다.
- ${e}^{-4}$ > (상대 오차)는 꺽임이 있는 목적함수에서는 괜찮다. 그렇지만 tanh 혹은 softmax를 쓰는 목적함수처럼 꺽임이 없다면 ${e}^{-4}$는 너무 크다.
- ${e}^{-7}$ 혹은 그보다 작은 상대 오차라면, 행복을 느껴야 한다.

하나 더 유념해야 할 것은, 망의 레이어 개수가 많아지면 상대 오차가 커진다. 이를테면 레이어 10개짜리 망에서 입력 데이터의 그라디언트를 체크한다면 <br />
에러가 층을 올라가며 축적되므로 ${e}^{-2}$정도의 상대 오차는 괜찮을 수도 있다. <br /> 
거꾸로 말하자면, 미분가능한 함수 하나만 갖고 노는데 ${e}^{-2}$ 의 상대 오차가 발생한다면 이것은 부정확한 그라디언트일 가능성이 매우 높다.

#### 이중 정확성 변수를 사용하라(Use double precision)

흔히들 실수하는 것이, 그라디언트 체크를 계산하는데 단일 정확성 부동 소수점 변수를 사용하는 경우가 있다. <br />
단일 정확성 변수를 쓰면 그라디언트 계산이 맞다하더라도 상대 오차가 ${e}^{-2}$정도로 커지는 경우가 종종 있다. <br />
경험상 이중 정확성 변수를 쓰면 상대 오차가 ${e}^{-2}$에서 ${e}^{-8}$까지 개선되는 경우도 있다.

#### 부동 소수점 연산이 확성화 되는 범위에서 계산하라

예를 들어, 신경망에서는 손실 함수를 배치별로 normalize하는 것이 보통이다. <br /> 
그렇지만 한 자료당 그라디언트가 매우 작다면, 거기에 또 데이터 갯수를 부가적으로 나눌 경우 매우 작은 수가 되고 더욱더 많은 수치적인 문제가 생길 수 있다. <br />
그래서 ${ f }_{ a }^{\prime }$와 ${ f }_{ n }^{\prime }$를 계속 찍어보고 두 값이 너무 작지 않은가를 확인하는 것이 좋다. <br />
(대충 ${e}^{-10}$)혹은 그보다 작은 크기의 값이면 걱정하지 않아도 된다) <br />
만약 두 값이 너무 작다면, 적당한 상수를 곱하여 부동소숫점 표현이 조금 더 괜찮도록(부동 소수점 표현에서 지수 부분이 0이 되도록) 만들 수도 있다.

#### 목적 함수에서의 꺽인 점

꺽인 점들에서 부정확한 계산이 발생할 수 있는데 이를 그라디언트 체크 과정에서도 염두에 두고 있어야 한다. <br />
꺽인 점은 목적 함수의 미분 불가능한 부분을 지칭하는 용어이다. ReLU함수(max(0,x)), 서포트 벡터 머신 목적함수나 맥스 아웃 뉴런 등을 사용하면 발생할 수 있다. <br />
꺽인 점이 야기시킬 수 있는 문제는 대략 다음과 같다. ReLU함수의 그라디언트를 $x=-{e}^{6}$에서 체크한다고 생각하여 보자. <br /> 
$x<0$이므로 ${ f }_{ a }^{\prime }$는 정확히 0이다. <br />
그렇지만, 수치적으로 계산된 그라디언트는 $f(x+h)$가 꺽인 점을 넘을 수도 있으므로 (이를테면 $h>{e}^{-6}$인 경우) 갑자기 0이 아닌 값을 내놓게 될 수도 있다. <br />
이런 경우까지 신경써야 하냐고 물을 수도 있는데, 사실 매우 흔하다. <br />
예를 들어, CIFAR-10을 위해 서포트 벡터 머신을 쓴다고 하면, 데이터가 50,000개 이고 한 데이터당 max(0,x) 항이 9개씩 있으니 결국 45만개의 ReLU항과 맞닥드리게 된다. <br />
게다가 서포트 벡터 머신 분류기와 신경망을 붙이면 ReLU들 때문에 꺽인 점이 더 늘어날 수도 있다.

#### 적은 수의 데이터만 써라

꺽인 점과 관련된 하나의 해결책은 더 적은 데이터를 쓰는 것이다. 손실 함수가 꺽인 점을 포함하고 있으면(ReLU나 Margin loss등을 사용한 경우) <br />
데이터가 적을수록 더 적은 꺽인 점을 포함할 것이고, 따라서 유한 차분 근사 과정에서 꺽인 점을 가로지르는 경우가 더 적을 것이다. <br />
게다가 2~3개의 데이터에 대해서만 그라디언트 체크를 수행하는 것이 거의 배치 전부에 대해 그라디언트를 체크하는 것이 될 테니 훨씬 빠르고 효율적이다. 

#### Step size h에 주의하라

꼭 작을수록 좋은것 만은 아닌게, h가 훨씬 작으면 수치적인 정확도 문제에 부딪힐 수 있다. <br />
가끔 그라디언트 체크가 잘 안 되면, h를 ${e}^{-4}$나${e}^{-6}$ 정도로 조정하여 보아라. 갑자기 잘 될수도 있다.

#### 특징적인 연산이 수행되는 곳에서 그라디언트 체크를 해라

그라디언트 체크는 파라미터 공간의 특정한(보통 랜덤인) 점 위에서 수행됨을 기억하자. <br />
그라디언트 체크가 한 점에서는 성공한다 하여도 다른 점에서 맞게 수행되리라고 믿기 힘들다. <br />
게다가, 초기값을 랜덤하게 줄 경우 그 점은 파라미터 공간의 가장 특징적인 점이 아닐 수도 있고, 분명 제대로 코딩된 듯한 그라디언트가 <br /> 
사실 잘 계산되지 않은 병적인 상황을 야기할 수도 있다. 예를 들어, SVM에서 초기 웨이트값을 매우 작게 설정하면, <br /> 
모든 데이터 포인트에 거의 0에 근접한 점수를 부여할 것이고 그라디언트 값들 또한 모든 데이터에 걸쳐 어떤 패턴을 나타낼 것이다. <br />
만약 그라디언트 구현이 잘못되었다면 이 패턴을 계속 만들어낼 것이고 좀 더 특징적인 계산으로(e.g: 몇 몇 점수가 다른 것보다 큰 경우) 일반화하지 못할 수도 있다. <br />
그러므로 안전하게 가려면, 네트워크가 학습을 시작할 무렵 짧은 Burn-in을 이용하고, 손실이 하강하기 시작한 뒤에 그라디언트 체크를 수행하는 것이 최선이다. <br />
요컨데, 첫 번째 반복에서부터 그라디언트 체크를 수행하면 그 때만의 병적인 오류 때문에 우리가 정말로 정확하게 그라디언트 체크를 수행하는 부분에서의 오류를 놓칠 수도 있다.

#### 정규화가 데이터를 압도하게 하지 마라

가끔, 손실 함수는 데이터 손실과 정규화 손실(e.g: 가중치값들에 대한 L2벌점)의 합으로 이루어져 있다. <br />
하나 알고 있어야 하는 위험은, 정규화 손실이 데이터 손실을 압도할 수 있다는 것인데, 이 경우 그라디언트는 주로 정규화 항에서 올 것이다. <br />
이 경우 데이터 손실 그라디언트가 올바르게 구현되지 못하는 상황을 감출수도 있다. <br />
그러므로, 먼저 정규화를 끄고 데이터 손실 부분만 체크를 수행하길 추천하며 그 다음에 정규화 항을 따로 점검해 보라. <br />
정규화 항만 따로 어떻게 점검 하는가 묻는다면, 하나의 방법은 코드를 해킹하여 데이터 손실 부분을 제거하는 것이다. <br />
다른 방법으로는 장규화 항의 강도를 높여서 그 효과가 그라디언트 체크 수행시 무시할 수 없게 키운 뒤 정규화 항 부분에서의 잘못된 그라디언트가 감지되도록 하여라.

## 학습 전에 제대로 돌아가는지 확인하는 팁과 트릭들

풀려는 최적화 문제가 매우 비싸(expensive)지기 전에, 다음 절차들을 돌려볼 만하다.

- 맞는 손실함수를 찾아라

적은 수의 파라미터로 초기화할 때는 당신이 기대한 손실 함수 값을 얻는지 확인하라. <br />
먼저 데이터 손실 함수 하나만 확인하는 것이 가장 낫다. (따라서 정규화 강도는 0으로 설정하여라) <br />
예를 들어, CIFAR-10에 Softmax 분류기를 이용할 경ㅇ 초기 손실 함수값을 2.302로 기대할 수 있는데, <br />
왜냐하면, $-ln(0.1)=2.302$가 각 클래스에 확률이 0.1로 분산되었을 테고 Softmax 손실 함수는 올바른 분류 확률에 음의 로그를 취한 값이기 때문이다. <br />
Weston Watkins SVM을 사용할 경우에는, (모든 스코어가 어림잡아 0이기 때문에) 고려되는 모든 마진값이 위반될테니 9의 손실값을 기대할 수 있다. <br />
(마진은 각각 잘못 분류된 클래스마다 1이다) 이런 손실값들이 나오지 않으면 초기화에 문제가 있을 수 있다.

- 두 번째 확인 절차로써, 정규화 강도를 올릴 수록 손실 함수값이 올라가야 한다.

- 자료의 작은 부분집합으로 과적합해 보라

마지막으로 가장 중요한 사항인데, 전체 데이터셋으로 훈련을 시작하기 전에, 작은 부분으로 훈련을 시도하여 보고(한 20개 정도), 0의 비용(Cost)을 달성 할 수 있는지 확인하여 보라. <br />
이 실험에서도 역시 정규화 강도는 0으로 설정하는 것이 가장 나으며, 그렇지 않으면 0의 비용을 얻을 수 없을 것이다. <br />
적은 자료에서의 이러한 확인 과정이 제대로 끝나지 않으면 전체 데이터셋으로 나아가는 것은 무가치하다. <br />
하나 강조할 것은, 아주 작은 데이터 셋에 성공적으로 과적합 하였지만, 여전히 코딩이 올바르게 이루어지지 않았을 수 있다. <br />
예를 들어, 가지고 있는 데이터 포인트들의 특성들이 어떤 버그 때문에 임의로 선정된 경우, <br /> 
작은 훈련 집합에서의 과적합은 성공일지 몰라도 그게 전체 데이터셋으로 일반화되지 않을 수도 있다.

## 학습 과정 돌보기

신경망을 훈련하는 중에 몇몇 쓸모있는 값은 모니터링해야 한다. 이런 도표들은 학습 과정을 지켜보는 창문이다. <br />
좀더 효율적인 학습을 위한 하이퍼 파라미터의 조정도 여기서 직관적 영감을 얻는다.

도표의 x축은 언제나 에폭(epoch)을 단위로 한다. 에폭은 각 자료가 몇 번이나 학습에 사용되었는가를 재는 용어이다. <br />
이를 테면 1 에폭이 지났다는 것은 모든 자료가 한 번씩 SGD iteration에 사용되었음을 뜻한다. <br />
x축으로 SGD 알고리즘 반복 횟수를 할 수도 있겠지만 에폭이 더 선호되는 편이다. <br />
반복 횟수는 배치 사이즈의 선택에 따라 임의로 바뀔 수 있기 때문이다.
___

#### 손실 함수

 <img src="./Images/4.png" width=700 />

좌측: 훈련 과정에서 학습 속도의 영향, 낮은 학습 속도로는 선형적인 향상이 이루어질 것이다. 높은 학습 속도에서는 좀 더 자수적인 향상이 보일 것이다. <br />
더 높은 학습 속도는 손실의 감소를 가속할 것이나, 더 나쁜 손실값에 빠지게 할 수도 있다.(초록선) <br />
그 이유는 최적화에 너무 많은 에너지가 가해져서 파라미터값들이 호돈스러운 형태로 움직이고(최적화 목적 함수 모양에서) 좋은 곳에 정착하기 힘들어지기 때문이다. <br />
우측: 전형적인 손실 함수의 예, x축은 시간(epoch)이고 CIFAR-10 데이터셋에서 작은 신경망을 훈련하였다. 이 손실 함수의 모양은 적절해 보이고, 
(손실 감소의 속도를 보았을때, 학습 속도가 작은 감은 있다) 배치 사이즈는 너무 작은 것으로 보인다.(비용(cost)에 노이즈가 너무 많다)
___

손실 함수의 "씰룩거림"은 배치 사이즈와 연관이 있다. 만일 배치 사이즈가 1이면 훨씬 더 많이 씰룩거릴 것이다. <br />
만일 배치 사이즈가 전체 데이터셋이면 이 씰룩거림은 최소화 될 것인데, 왜냐하면 모든 그라디언트 업데이트가 손실함수를 단조적으로 향상시킬 것이기 때문이다. <br />
(학습 속도가 너무 크지만 않다면) <br />
어떤 사람들은 손실 함수로 로그값의 그래프를 선호하기도 한다. <br /> 
일반적으로 학습 과정은 어떤 지수적인 모양을 취하고 있기 때문에, 로그 손실 그래프틑 좀 더 해석이 용이한 지선의 모양처럼 보인다. <br />
부가적인 사항으로, 만약 여러 개의 교차 검증 모형의 손실 그래프를 같은 그래프 위에 그리면, 그들 사이의 차이가 좀 더 명백해지는 장점이 있다.

#### 훈련/검증 정확도

훈련/검증 정확도는 분류기 훈련시 추적해야 할 또다른 중요한 값이다. 이 플롯은 당신의 모형이 과적합 중인지 발견할 수 있는 값진 인사이트를 제공한다.
___

<img src="./Images/5.png" width=400 />

훈련/검증 정확도의 차이는 오버 피팅의 정도를 가리킬 수 있다. <br />
위 그림에서, 파란색(검증 오류)곡선은 훈련 정확도에 비하여 매우 낮은 검증 정확도를 보여주고 있는데, 이는 강한 과적합의 가능성을 시사한다. <br />
(어떤 지점 이후에 검증 정확도가 갑자기 떨어질 수도 있는 것도 가능하다) <br />
실제로 당신이 이 현상을 보게 되면 아마 정규화를 쓰거나 (더 강한 L2벌점(Penalty)나 드랍아웃 등...)데이터를 더 모으고 싶을 것이다. <br />
다른 가능성으로는 검증 정확도가 훈련 정확도를 꽤 잘 따라가는 것이다. <br />
이것은 당신의 모델의 수용량이 충분히 높지 않음을 시사할 수도 있다. 파라미터(웨이트)의 개수를 늘려서 모형을 더 크게 만들어 보아라.
___

#### 웨이트의 현재 값과 변화량의 비율

마지막으로, 웨이트의 현재 크가와 업데이트로 인한 변화량의 크기를 비교해 볼 수도 있다. <br /> 
(Note: 그냥 날 것의 그라디언트 값이 아니라, 웨이트의 변화량이다(이를 테면, vanilla SGD에서는 학습 속도와 그라디언트의 곱이다)) <br />
모든 파라미터의 집합마다 독립적이로 이 비율을 추적/계산하고 싶은가? <br />
이 비율은 ${e}^{-3}$근처이어야 한다. 이보다 낮으면 학습 속도가 너무 낮은 것이다. 이보다 크다면 학습 속도가 너무 크다. <br /> 
특정한 예를 들자면 아래와 같다.

In [ ]:
# assume parameter vector W and its gradient vector dW
param_scale = np.linalg.norm(W.ravel())
update = -learning_rate*dW # simple SGD update
update_scale = np.linalg.norm(update.ravel())
W += update # the actual update
print update_scale / param_scale # want ~1e-3

최소값이나 최대값을 추적할 수도 있고, 그라디언트와 업데이트값의 Norm을 계산하고 추적할 수도 있다. 이 지표들은 대게 연관성이 높아서 거의 비슷한 결과를 준다.

#### 층별 활성값 및 그라이언트의 분포

올바르지 않은 초기값 설정은 학습 과정을 느리게 하거나 완전히 말칠 수 있다. 운 좋게도 이 이슈는 상대적으로 쉽게 분석할 수 있다. <br />
한 방법은 활성값/ 그라디언트값의 히스토그램을 망의 모든 층마다 그려보는 것이다. <br />
직관적으로 생각해보면 만일 이상한 분포가 나오면 좋은 징조가 아닐 수 있다. 이를테면, tanh뉴런에서는 활성값이 [-1,1]의 전 점위에 걸쳐 분산되어 있는 모습을 보고 싶다. <br />
혹시 모든 활성값이 0을 내놓거나 -1 혹은 1에 집중되어 있으면 문제가 있는것이다.

#### 첫 번째 층의 시각화

마지막으로, 만일 당신이 이미지 픽셀에 관련된 일을 한다면 첫 층의 특징들을 시각화하는 것이 많은 도움이 될 수도 있다.
___

<img src="./Images/6.png" width=700 />

신경망의 첫 층의 웨이트값을 시각화한 예 <br />
좌측: 특징값에 잡음이 많을 때 나타날 수 있는 증상. 수렴하지 않은 망, 적절하지 않은 학습 속도, 매우 낮은 정규화 패널티 <br />
우측: 부드럽고 깨끗하며 다양한 피처값들이 보이는 경우 훈련이 잘 진행되고 있다는 지표일 수 있다.
___

## 파라미터값의 업데이트

수식적으로 그라디언트값은 역전파로 계산되고 이는 파라미터값 업데이트를 위해 사용된다. 업데이트를 수행하는 몇 접근법들이 있는데 뒤에 서술하겠다.

딥 네트워크에서의 최적화 문제는 지금 가장 활발히 연구가 진행되고 있는 분야이다. 이 섹션에서는 공통적으로 자주 쓰이는 테그닉과 그것들의 직관적인 아이디어를 살펴본다. <br />
디테일한 사항은 수업의 범위를 넘으므로 다루지 않겠다.

#### SGD와 그 외

- 바닐라 업데이트(Vanilla Update)

가장 간단한 업데이트 형태는 그라디언트의 반대방향으로 파라미터를 업데이트 하는 것이다. 왜냐하면 그라디언트는 증가하는 방향을 가리키기 때문이다. <br />
그렇지만 우리는 손실 함수를 최소화하고 싶어한다. 파라미터의 벡터를 x라 하고 그라디언트를 dx라 쓰면, 가장 간단한 업데이트는 다음과 같다.

In [ ]:
# Vanilla update
x += - learning_rate * dx

여기서 학습속도 learning_rate는 하이퍼 파라미터이고 고정된 상수이다. <br /> 
만일 dx가 전체 데이터셋에서 계산되고 학습 속도가 충분히 작을 때, 최소한 나쁜 프로세스는 아님을 보장한다.

- 모멘텀 업데이트(Momentum Update)

이것은 적어도 딥 네트워크에서는 바닐라 업데이트보다 더 잘 수렴한다 이 방법은 최적화 문제를 물리학적 관점에서 바라보는데서 유래했다. <br />
자세히 말하자면, 손실 함수는 구릉지대에서 높이에 해당한다.(그래서 포텐셜 에너지에도 대등되는데 $U=mgh$이고 따라서 $U\propto h$이다) <br />
파라미터의 초기값을 임의로 정하는 것은 입자를 어떤 위치에서 0의 속도로 세팅하는 것과 똑같다. <br />
이 상황에서 최적화 과정은 파라미터 벡터(즉 입자)를 굴리는 과정과 동일하다고 볼 수 있다.

입자에 작용하는 힘은 포텐셜 에너지의 그라디언트(즉 $F=--\nabla U$)와 관련이되어 있으므로, 입자가 느끼는 힘은 정확하게 손실 함수의 그라디언트의 반대 부호이다. <br />
$F=ma$이므로 그 그라디언트(의 반대 부호)는 입자에 작용하는 가속도에 비례한다. 위에서 SGD와 다른 점을 발견했는가? <br />
SGD는 위치값(현재 파라미터값)에 그라디언트가 직접 합쳐진다. <br /> 
모멘텀 업데이트는, 물리학적 관점에서, 그라디언트가 오직 속도에만 직접적으로 영향을 주고 속도가 위치값에 영향을 줄 것을 제안하고 있다.

In [ ]:
# Momentum update
v = mu * v - learning_rate * dx # integrate velocity
x += v # integrate position

여기서 우리는 새로운 변수 v를 도입하고 0으로 초기화했다. mu는 또 하나의 하이퍼 파라미터이다. 정확한 용어는 아니지만 우리는 이 mu를 모멘텀(운동량)이라 부르기로 한다. <br />
(보통 0.9로 설정한다) 사실 마찰 계수라고 부르는 쪽이 더 mu에 맞기는 하다. 이 변수는 입자의 현재 속도 및 운동에너지를 효과적으로 감소시키도록 도와준다. <br />
이게 없다면 아마 입자는 언덕의 아래쪽에 절대 멈추지 못할 것이다. 만약 모멘텀을 교차 검증으로 선택한다면 보통 [0.5, 0.9, 0.95, 0.99]로 설정한다. <br />
에폭에 따라 모멘텀의 크기를 조정하면 최적화에 더 이로울 수도 있다. 이를테면 시작할 때는 0.5의 모멘텀으로 시작하되 몇 번의 에폭을 지나면 0.99로 설정할 수도 있다. <br />
이는 학습 속도의 스케줄을 담금질하는 것과도 비슷하다.(뒤에 다시 논의할 예정)

> 모멘텀 업데이트를 쓰면, 파라미터 벡터가 업데이트되는 속도의 방향은 그라디언트들이 많이 향하는 방향으로 축적될 것이다.

#### Nesterov 모멘텀

최근에 많은 주목을 받은 Nesterov 모멘텀은 모멘텀 업데이트와 조금 다르다. 볼록 함수에서는 이 업데이트가 강력한 이론적 성질을 갖고 있고, <br /> 
실제 상황에서도 보통의 모멘텀 방법론보다 조금 더 낫다고 한다.

핵심 아이디어는 다음과 같다. 만약 현재 파라미터 벡터가 $x$라는 어떤 위치에 있다고 치고 위의 모멘텀 업데이트를 보자. <br />
만일 위의 Intergrate velocity 과정에서 뒷항 없이 $v = mu*v$만 있다고 가정하며느 다음 위치로 $x + mu*v$가 예견될 것이다. <br />
그러므로 이전의 오래된 위치 $x$대신 예견된 위치 $x + mu*v$에서 그라디언트를 계산하는 것이 합리적일 수 있다.
___

<img src="./Images/7.png" width=800 />

Nestrov 모멘텀 <br /> 
지금 위치(붉은색 원)에서 모멘텀에 의해 연두색 화살표이 끝점으로 이동할 상황이다. <br />
Nestrov 모멘텀은 현재 위치에서 그라디언트를 계산하는 것이 아니라 이 예견된 위치(화살표 끝점)에서 그라디언트를 계산한다.
___

다른 말로 하면, 다음과 같이 계산한다.

In [ ]:
x_ahead = x + mu * v
# evaluate dx_ahead (the gradient at x_ahead instead of at x)
v = mu * v - learning_rate * dx_ahead
x += v

실제 용례에서 사람들은 위 식을 재서술하여 바닐라 SGD나 이전의 모멘텀 업데이트의 꼴처럼 고칠 때가 있다. <br />
이를 테면 x_head = x + mu * v 부분을 손보고, 업데이트를 x의 관점이 아닌 x_head의 관점에서 서술하면(그리고 x_head를 x로 고쳐쓰면) 아래와 같다. <br />
사족을 달면 이제 우리가 저장하는 파라미터 벡터는 언제나 예견된 버전이다.

In [ ]:
v_prev = v # back this up
v = mu * v - learning_rate * dx # velocity update stays the same
x += -mu * v_prev + (1 + mu) * v # position update changes form

## 학습 속도 담금질

깊은 신경망의 훈련에서 시간에 따라 훈련 속도를 담금질(조정)하는 건 언제나 도움이 된다. <br />
이 직관을 기억해 두면 도움이 된다. 높은 학습 속도에서는 전체 시스템이 너무 높은 운동 에너지를 갖고 있어서 파라미터 벡터가 혼돈스럽게 튀고, <br />
손실 함수의 좁고 깊숙한 골짜기 안으로 쏙 들어가서 정착하기 힘들다. <br />
그러면 학습 속도를 언제 줄일 것인가? 우선 천천히 줄여봐라. 그러면 오랜 시간동안 거의 제자리에서 혼돈스럽게 왔다 갔다 할 것이다. <br />
그렇지만 너무 빨리 줄이면 전체 시스템이 너무 빨리 식을 것이고, 갈 수 있는 최적의 장소에 도달하지 못 할 수도 있다. <br />
학습 속도를 감소시키는 방법은 보통 다음 세 가지가 있다.

- 계단식 감소 <br />
몇 에폭마다 일정량 만큼 학습 속도를 줄인다. 전형적으로는 5에폭마다 반으로 줄아거나 20에폭마다 $\frac { 1 }{ 10 } $씩 줄이기도 한다. <br />
이 숫자들은 전적으로 문제와 모형의 타입에 의존한다. <br /> 
실전에서는, 우선 고정된 학습 속도로 검증 오차를 살펴보다가 개선되지 않을 때마다 학습 속도를 감속시키는 방법을 택하기도 한다.

- 지수적 감소 <br />
$\alpha ={ \alpha  }_{ 0 }{ e }^{ -kt }$꼴을 뜻한다. 여기서 ${ \alpha  }_{ 0 }$, $k$는 하이퍼 파라미터이고 $t$는 반복 횟수이다. (물론 에폭을 단위로 해도 된다)

- $\frac { 1 }{ t } $ 감소 <br />
$\alpha =\frac { { \alpha  }_{ 0 } }{ 1+kt } $꼴을 뜻하고 여기서 ${ \alpha  }_{ 0 }$, $k$는 하이퍼 파라미터이고 $t$는 반복 횟수이다.

실전에서는 계단식 감소 방식이 조금 더 선호될만 한데, 관련된 하이퍼 파라미터들(몇 에폭마다 감소시킬지, 감소율)이 k에 비해서 해석이 더 쉽기 때문이다. <br />
마지막으로, 계산 자원이 충분하다면 감소율을 좀 더 낮춰서 오랜 시간동안 모형을 훈련시켜라.

## 이차 근사 방법들

딥러닝 맥락에서 두 번로 대중적인 최적화 방법은 뉴턴 방법인데 다음과 같은 업데이트 방식을 뜻한다.

<img src="./Images/10.png" width=300 />

여기서 $Hf(x)$는 헤시안 행렬로, 다변수 함수의 2차 미분으로 이루어진 정방행렬을 뜻한다. $\nabla f(x)$항은 (그라디언트 감소에서 보았던) 그라디언트 벡터이다. <br />
직관적으로 헤시안 행렬은 어떤 함수의 국지적인 곡률을 뜻하고 이 정보로 우리는 더 효율적인 업데이트를 수행할 수 있다. <br />
특별히, 헤시안 행렬의 역행렬을 곱함으로써, 휨이 약한 방향으로는 더 공격적으로 그리고 휨이 강한 방향으로는 짧게 움직일 수 있다. <br />
일파 근사 방법에 비해 뉴턴 방법이 가지는 강점은, 위의 엡디이트 공식을 보면 학습 속도에 대한 하이퍼 파라미터가 없다는 것이다.

그렇지만 위의 업데이트는 거의 모든 실제 상황에서는 쓸모가 없는게, 공식 그대로 헤시안 행렬을 계산한다면(역행렬을 취하는 일을 포함하여) 상상도 못할 시간과 메모리가 필요하다. <br />
예를 들면, 파라미터가 백만개 정도인 신경망은 [10,000,000 x 10,000,000]크기의 헤시안 행렬을 필요로 하고 이는 3725GB의 램을 필요로 한다. <br />
그 결과로 다양한 유사-뉴턴 방법이 역-헤시안 행렬을 근사하기 위해 고안되었다. 이 방법론들중 L-BFGS가 가장 대중적이다. <br />
L-BFGS는 시간에 따른 그라디언트의 변화를(간접적으로) 근사에 이용한다. 즉, 전체 행렬은 절대 계산되지 않는다. <br />

그렇다고 해도, 메모리 걱정을 없앤다고 할지라도, L-BFGS를 그냥 적용하자면 큰 단점이 하나 있는데 바로 전체 훈련 집합 전체를 대상으로 계산하여야 한다는 점이다. <br />
수 백만 개체가 있는 그 데이터셋 말이다. 배치-SGD와는 달리, 미니배치에서 L-BFGS가 작동하게 하는 방법은 좀 더 꼼수를 필요로 하며 활발한 연구 분야이다. 

실제 상황에서 지금까지는 L-BGFS나 다른 이차 근사 방법이 대규모 딥러닝이나 CNN에서 사용되지 않는 게 보통이다. <br /> 
표준적으로는 SGD와 그 변종들(모멘텀이나 Nesterov's 모멘텀)이 훨씬 간단하고 계산도 빨라서 많이 사용된다.

## 파라미터별 데이터-맞춤 학습 속도

지금까지 논의된 접근법들은 모든 파라미터에 똑같은 학습 속도를 적용하였다. <br />
학습 속도의 튜닝은 계산이 많은 작업인지라, 데이터에 맞추어 자동으로 학습 속도를 정하는 방법을 찾고자 많은 사람들이 노력하였다. <br />
파라미터별로 학습 속도를 다르게 하고 이를 데이터-맞춤으로 정하려는 노력들 또한 있었다. <br />
이러한 방법들은 보통 또다른 하이퍼 파라미터 세팅이 필요하긴 하지만, 이 하이퍼 파라미터는 넓은 범위에서 잘 작동하는 편이라 일반적인 학습 속도 튜닝보다는 덜 까다롭다. <br />
이번 절에서는 실전에서 마주칠 수도 있는 주요 데이터-맞춤 방법들을 조망해본다.

- Adagrad: 데이터-맞춤 학습 속도 조정 방법 중 하나이다. 

In [ ]:
# Assume the gradient dx and parameter vector x
cache += dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)

위에서 변수 cache는 그라디언트 벡터의 사이즈와 동일한 사이즈를 갖고 있다. <br />
cache의 각 성분은(해당 성분에 대응하는) 그라디언트의 제곱값들을 계속 추적하고 있고, 파라미터 업데이트에서 성분별로, 일종의 표준화 기능을 수행한다. <br />
주목할 점은, 높은 그라디언트 값을 갖는 웨이트값들은 점점 실질적인 학습 속도가 감소하고, 그라디언트 값이 낮거나 업데이트가 거의 없는 웨이트 값들은 실질 학습속도가 증가한다는 것이다. <br />
놀랍게도 제곱근 연산이 여기서 중요한 비중을 차지한다. 제곱근이 없다면 알고리즘의 성능이 많이 나빠진다. <br />
변수 eps는 분모가 너무 0에 가깝지 않도록 안정화 역활을 하고 주로 ${e}^{-4}$에서 ${e}^{-8}$의 값이 할당된다. <br /> 
Adagrad의 단점이 있다면, 딥러닝의 경우에는, 학습 속도가 단조적이라 너무 한 방향으로 급진적으로 나가거나, 혹은 학습을 너무 빨리 멈출 가능성도 있다.

- RMSprop

RMSProp은 매후 효과적이다. RMSProp 업데이트는 Adagrad를 간단히 조정하여 급진적이고 단조 감소하는 학습 속도를 경감시켰다. <br />
제곱 그라디언트의 평균이 아니라 이동 평균(Moving average)를 사용한다.

In [ ]:
cache = decay_rate * cache + (1 - decay_rate) * dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)

여기서 decay_rate는 하이퍼 파라미터이고 보통 [0.9, 0.99, 0.999]중 하나의 값을 취한다. <br />
주목할 점은 += 업데이트는 Adagrad와 동등하지만, cache가 어디선가 샌다. <br /> 
따라서 RMSProp은 여전히 각 웨이트 값을 그것의 과거 그라디언트 값으로 조정하여 성분별로 실질 학습 속도를 비슷하게 만드는 효과를 갖고 있지만, <br />
Adagrad처럼 학습 속도가 단조적으로 줄지는 않는다. 

- Adam

이것은 비교적 최근에 제안된 방법인데, RMSProp에 모멘텀을 혼합한 것처럼 보인다. 간단하게 쓰면 업데이트는 다음과 같다.

In [ ]:
m = beta1*m + (1-beta1)*dx
v = beta2*v + (1-beta2)*(dx**2)
x += - learning_rate * m / (np.sqrt(v) + eps)

업데이트는 RMSProp의 업데이트 방식과 정확히 같아 보이는데, 그냥 노이즈가 껴있을 수도 있는 그라디언트 dx대신에 안정화된 버전인 m이 사용되었다는 점이 다르다. <br />
추천되는 하이퍼 파라미터 값들은 eps=1e-8, beta1=0.9, beta2=0.999이다. 실전에서 Adam은 기본 알고리즘으로 추천되고 있고, 가끔은 RMSProp보다 조금 더 잘 하기도 한다. <br />
그러나 SGD+Nesterov Mementum도 대안으로 해볼만 하다. <br /> 
Adam 업데이트 절차에는 편향 보정(Bias Correction) 매커니즘이 반영되어 있는데, 벡터 m, v가 나중에 완벽하게 "워밍 업" 되기 전에 (Iteration의 처음 몇 스텝에서) <br />
초기화되어 0에 편향되어 있다는 점을 보상하기 위해서이다. 업데이트 코드는 다음과 같다.

In [ ]:
# t is your iteration counter going from 1 to infinity
m = beta1*m + (1-beta1)*dx
mt = m / (1-beta1**t)
v = beta2*v + (1-beta2)*(dx**2)
vt = v / (1-beta2**t)
x += - learning_rate * mt / (np.sqrt(vt) + eps)

업데이트는 이제 다른 파라미터들 처럼 반복의 함수이다.
___

<img src="./Images/8.png" width=800 />

왼쪽: 손실 함수의 등고선 위에서 각 최적화 알고리즘들의 시간(Iteration)에 따른 변화. 모멘텀 기반 방법론들의 급가속 행동들에 주목하라. <br />
이게 최적화를 마치 언덕을 내려가는 고어럼 보이게 만든다. <br />
오른쪽: 목적 함수에 안장점(Saddle Point)가 있을 때의 시각화. 안장점은 그라디언트가 0이지만 헤시안 행렬의 고유치(Eigenvalue)에 양수/음수가 섞여있을 때 발생한다. <br />
SGD는 안장점을 빠져나오는데 매우 힘든 시간을 겪는다. <br /> 
반대로, RMProp같은 알고리즘들은 안장의 방향으로 매우 작은 그라디언트를 마주하게 되지만 분포-표준화 성질 때문에 이 방향의 실질 학습 속도를 높일 수 있고 <br />
따라서 이 방향으로 빠져나올 수 있다.
___

## 하이퍼 파라미터 최적화

일전에 본 대로, 신경망의 훈련에는 많은 하이퍼 파라미터 설정이 관련된다. 신경망 관련 논의에서 가장 빈번하게 등장하는 하이퍼 파라미터는 다음과 같다.

- 학습 속도의 초기값
- 학습 속도의 경감 계획, 이를 테면 경감 상수
- L2나 드랍아웃 패널티의 정규화 강도

그렇지만 역시 본 대로, 덜 민감한 하이퍼 파라미터들도 있는데, 이들은 파라미터별 데이터-맞춤 학습 방법, 모멘텀 관련 스케쥴등에서 등장하였다. <br />
이번 절에서는 하이퍼 파라미터를 수행하기 위한 추가적인 팁이나 트릭들을 언급한다.

#### 코드 구성 단계에서

큰 신경망은 대게 긴 학습시간이 걸리고, 따라서 하이퍼 파라미터 검색에는 몇일, 몇 주가 걸릴 수도 있다. 코드를 짤 때 이 점을 염두에 두는 것이 중요하다. <br />
하나 가능한 코드 구성은, 하이퍼 파라미터를 임의로 선택하여 최적화를 수행하는 일꾼을 만드는 것이다. <br />
이 일꾼에게 훈련과정에서 매 에폭 뒤의 검증 성능을 쭉 추적하여 모형의 체크 포인트들을 (다른 훈련 통계량등, 이를테면 시간에 따른 손실 함수값들과 함께) <br />
파일에 저장케 하라. 공유 파일 시스템위에 저장하면 더 좋다. 검증 성능을 아예 직접 파일 이름에 써 놓는 것도 좋다. 그러면 과정이 더 단축되고 단순할 것이다. <br />
그리고 마스터라 불릴 두 번째 프로그램을 만들어서 계산 클러스터별 일꾼들을 개시하거나 끝내게 하라. <br />
혹은 마스터는 일꾼이 작성한 체크 포인트들을 조사하고 훈련 통계량들로 그림을 그릴 수도 있다.

#### 교차 검증보다는 단일한 검증 집합

많은 경우에, 적당한 크기의 검증 집합을 설정해 두어 한 번만 검증하는 것이, 여러 번의 교차검증보다 코드를 단순화 시킨다. <br />
사람들이 교차 검증을 했다고 얘기해도, 많은 경우에 그 사람들은 단일한 검증 집합만 썻을 것이다. 

#### 하이퍼 파라미터의 범위

로그 스케일로 하이퍼 파라미터를 찾아라. 예를 들어, 학습 속도의 선정은 전형적으로 다음과 같아 보일 수도 있다. <br />
learning_rate = 100 ** uniform(-6, 1). 다시 말하면, 균등 분포에서 난수를 뽑은 뒤에 이를 10의 제곱값으로 취하는 것이다. <br />
같은 전략이 정규화 강도 검색에도 사용되어야 한다. 왜나면, 직관적으로, 학습 속도와 정규화 강도는 학습 동역학에 배수적인 효과가 있기 때문이다. <br />
학습 속도는 업데이트에서 그라디언트에 곱해지는 수이다. 이를 테면, 최초 학습 속도가 0.001이면 이를 계속 더할 경우 동역학에 큰 영향을 미치지만, <br />
최초 학습 속도가 10인 경우에는 거의 영향이 없다. 그러므로 학습 속도의 범위는 어떤 값을 계속 곱하거나 나누는 것이 빼거나 더하는 것보다 더 자연스럽다. <br />
대신에, 어떤 하이퍼 파라미터들이(이를테면 드랍아웃)은 보통의 스케일에서 검색된다. (예, dropout = uniform(0, 1))

#### 그리드 검색보다는 임의 검색

임의로 선택된 하이퍼 파라미터들이 최적화를 통해 그리드에서 선택된 하이퍼 파라미터들보다 더 효과적이다.
___

<img src="./Images/9.png" width=600 />

어떤 하이퍼 파라미터는 다른 것보다 훨씬 중요할 때가 많다.(예: 오른쪽 그림에서 꼭대기에 있는 하이퍼 파라미터 vs 왼쪽 그림) <br />
그리드 검색보다는 임의 검색이 좋고 중요한 하이퍼 파라미터 발견을 더 용이하게 한다

#### 가장 좋은 값이 경계에 있으면 조심하라

가끔은 하이퍼 파라미터 검색 범위(이를테면, 학습 속도)가 나쁘게 설정되었을 수도 있다. 예를 들어, learning_rate = 10 ** uniform(-6, 1)을 사용한다고 가정하여 보자. <br />
한번 결과를 받았으면, 최종 학습 속도가 이 구간의 끝에 있지 않아야 한다. 그렇지 않으면, 우리는 구간 밖에 있는 더 최적의 하이퍼 파라미터를 놓치고 있는지도 모른다.

#### 성긴 검색에서 촘촘한 검색으로

실전에서는, 처음에는 널찍한 범위에서 검색을 하다가 (예: 10 ** [-6, 1]), 좋은 결과가 어디에서 발생하느냐에 따라 범위를 좁힐 수도 있다. <br />
또한, 처음의 성긴 검색에서는 1에폭이나 혹은 더 적게만 훈련하는게 도움이 될 수도 있는데, 왜냐하면 많은 하이퍼 파라미터 세팅에서는 하나도 학습하는게 없을 수도 있고, <br />
또는 무한대의 손실 함수값으로 폭발할 수도 있기 때문이다. 두 번째 단계는 좀 더 좁은 범위에서의 검색을 5에폭 정도로 할 수 있을 것이다. <br />
그리고 마지막 검색에서는 좁은 범위에서 많은 에폭의 훈련을 수행해도 좋겠다.

#### 베이지안 하이퍼 파라미터 최적화

하이퍼 파라미터를 좀 더 효율적으로 항해하는 방법을 고안하기 위한 분야이다. <br />
핵심 아이디어는 하이퍼 파라미터들의 성능을 평가할 때 탐험 - 개발의 상충에서 적절한 균형을 찾는 것이다. <br />
그러나 아직 ConvNet관련된 실전 세팅에서는 임의 검색이 상대적으로 더 뛰어나다

## 모형 앙상블

실전에서, 신경망의 성능을 몇 퍼센트 끌어올릴 수 있는 믿을 만한 방법이 하나 있는데, 바로 여러 개의 독립적인 모형을 만들고 테스트 때 그들의 평균 예측을 취하는 것이다. <br />
앙상블에 관여하는 모형이 많아지면, 보통 성능은 단조적으로 개선된다. 게다가, 앙상블 내에서 모형의 다양함이 늘어날수록 성능의 개선은 더 극적이다. <br />
아래는 앙상블을 구축하는 몇 가지 방법이다.

- 같은 모형 다른 초기화

교차 검증으로 최고의 하이퍼 파라미터를 결정한 다음에, 같은 하이퍼 파라미터를 이용하되 다른 초기값을 임의로 다양하게 여러 모형을 훈련한다. <br />
이 접근법의 위험은, 모형의 다양성이 오직 다양한 초기값에서만 온다는 것이다.

- 교차 검증 동안 발견되는 최고의 모형들

교차 검증으로 최고의 하이퍼 파라미터들을 결정한 다음에, 몇 개의 최고 모형을 선정하여 이들로 앙상블을 구축한다. <br />
이 방법은 앙상블내의 다양성을 증대하거나, 준-최적 모형을 포함할 수도 있는 위험이 있다. <br />
실전에서는 이를 수행하는게 위에 것들보다 쉬운 편인데, 교차 검증 뒤에 추가적인 모형의 재훈련이 필요없기 때문이다.

#### 한 모형에서 다른 체크포인트들

만약 훈련의 비용이 높다면, 어떤 사람들은 단일한 네트워크의 체크 포인트들을 (이를테면 매 에폭 후) 앙상블하여 제한적인 성공을 거둔 바 있음을 기억해두라. <br />
명백하게 이 방법은 다양성이 떨어지지만, 실전에서는 합리적으로 잘 작동할 수 있다. 이 방법은 매우 간편하고 저렴하다는 것이 장점이다. 

#### 훈련 동안의 모수값들에 평균을 취하기

훈련 동안(시간에 따른) 웨이트 값들의 지수 하강 합을 저장하는 제 2의 네트워크를 만들면 언제나 몇 퍼센트의 이득을 값싸게 취할 수 있다. <br />
이 방식으로 우리는 최근 몇 Iteration동안의 네트워크에 평균을 취한다고 생각할 수도 있다. <br />
마지막 몇 스텝 동안의 웨이트 값들을 이렇게 안정화시킴으로서 우리는 언제나 더 나은 검증 오차를 얻을 수 있다. <br />
직관적으로 생각하자면, 목적 함수는 볼-모양이고 우리의 네트워크는 극값 주변을 맴돌 것이므로, 평균을 취하면 극값에 더 가까운 어딘가에 다다를 기회가 더 많아질 것이다.

모형 앙상플의 단점이 하나 있다면 테스트 샘플에 모형을 적용할 때 평가에 더 시간이 걸린다는 점이다. 

## Summary

To train a Neural Network:

- Gradient check your implementation with a small batch of data and be aware of the pitfalls.
- As a sanity check, make sure your initial loss is reasonable, and that you can achieve 100% training accuracy on a very small portion of the data
- During training, monitor the loss, the training/validation accuracy, and if you’re feeling fancier, the magnitude of updates in relation to parameter values (it should be ~1e-3), and when dealing with ConvNets, the first-layer weights.
- The two recommended updates to use are either SGD+Nesterov Momentum or Adam.
- Decay your learning rate over the period of the training. For example, halve the learning rate after a fixed number of epochs, or whenever the validation accuracy tops off.
- Search for good hyperparameters with random search (not grid search). Stage your search from coarse (wide hyperparameter ranges, training only for 1-5 epochs), to fine (narrower rangers, training for many more epochs)
- Form model ensembles for extra performance

## Additional References

- [SGD](https://www.microsoft.com/en-us/research/publication/stochastic-gradient-tricks/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F192769%2Ftricks-2012.pdf) tips and tricks from Leon Bottou
- [Efficient BackProp](http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf) (pdf) from Yann LeCun
- [Practical Recommendations for Gradient-Based Training of Deep Architectures](https://arxiv.org/pdf/1206.5533v2.pdf) from Yoshua Bengio